# 06 Object References, Mutability, Recycling
Some notes, observations and questions along chapter 06.

- every Python object has an identity, a type, and a value
- only the value of an object may change over time
- remark: if I change the type of an object, it does get re-created as a new object (and thus gets a new identity and value, too)

### Variables Are Not Boxes
- but rather sticky notes to boxes (references)

In [1]:
a = [1, 2, 3]
b = a
a.append(4)
b  

[1, 2, 3, 4]

`dir()` provides a list of names available in the current namespace.
Without any argument it refers to the module. 
With an argument, it would return a list of local namespaces (meaning local to the object).

In [3]:
class Gizmo:
    def __init__(self):
        print(f'Gizmo id: {id(self)}')

x = Gizmo()

dir()

Gizmo id: 138667950609696


['Gizmo',
 'In',
 'Out',
 '_',
 '_1',
 '_2',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i2',
 '_i3',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'a',
 'b',
 'exit',
 'get_ipython',
 'open',
 'quit',
 'x']

### Identity, Equality, and Aliases
- objects with the same ID are the same
- objects with different IDs can still be alike

In [1]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
lewis is charles

True

In [2]:
id(charles), id(lewis)

(132560207303872, 132560207303872)

In [4]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832}
alex == charles

True

In [5]:
alex is charles

False

Python Language Reference: "An object’s identity never changes once it has been created; you may think of it as the object’s address in memory. The is operator compares the identity of two objects; the id() function returns an integer representing its identity."

### Choosing Between == and is
- `is` is used to compare something with 
    - a singleton (class can only have one instance) like `None` or with 
    - a sentinel (special, unique object used to indicate a specific condition or as a placeholder. It’s used to differentiate between different states or absence of a value.)
        - Sentinel objects are most useful where boundary conditions are complex.
- computing `is` is as simple as comparing two integer IDs

In [6]:
# example sentinel:

END_OF_DATA = object()
# ... many lines
def traverse(...):
    # ... more lines
    if node is END_OF_DATA:
        return
    # etc.

SyntaxError: invalid syntax (2484180808.py, line 5)

- `==` is used to determaine alike-ness between objects
- implementation differs
- a == b is syntactic sugar for a.__eq__(b)
- "The __eq__ method inherited from object compares object IDs, so it produces the same result as is. But most built-in types override __eq__ with more meaningful implementations that actually take into account the values of the object attributes."

### The Relative Immutability of Tuples
- "Tuples, like most Python collections—lists, dicts, sets, etc.—are containers: they hold references to objects.2 If the referenced items are mutable, they may change even if the tuple itself does not."

### Copies Are Shallow by Default
- shallow copy: outermost container is duplicated, but the copy is filled with references to the same items held by the original container; but if these contained objects are mutable, we most likely prefer deep copies

In [8]:
# shallow copy:
l1 = [3, [55, 44], (7, 8, 9)]
l2 = list(l1) # same: copy.copy(l1)
print(l2 == l1)
print(l2 is l1)

True
False


### Function Parameters as References
- passing parameters into functions is "call by sharing" (or "call by object reference"), meaning when passing arguments to a function in Python, we're passing references to the objects, not the actual objects themselves
- the parameters inside the function become aliases of the actual arguments
- a function may change any mutable object passed as a parameter, but it cannot change the identity of those objects
    - up until now I had regarded params as copies, but they are not!!!

In [11]:
def f(a, b):
    a += b
    return a

x = 1
y = 2
print(f(x, y))
# number x is unchanged:
x, y

3


(1, 2)

In [12]:
a = [1, 2]
b = [3, 4]
print(f(a, b))
# list a is changed:
a, b

[1, 2, 3, 4]


([1, 2, 3, 4], [3, 4])

In [13]:
t = (10, 20)
u = (30, 40)
print(f(t, u))
# tuple t is unchanged:
t, u

(10, 20, 30, 40)


((10, 20), (30, 40))

Integers and tuples a immutable. When we perform a += b with integers or tuples, Python creates a new integer or tuple that is the concatenation of a and b.

Lists on the other hand are mutable in Python. When we perform a += b, it modifies the list a in place by appending the elements of b to a.

This is the same as everywhere else: If we modify a passed object through one of these param aliases, the change will be reflected outside the function because the function operates on the same object.

#### Mutable Types as Parameter Defaults: Bad Idea
We should avoid mutable objects as default values for parameters.

If a default value is a mutable object, and we change it, the change will affect every future call of the function. The same object is used over when the function is called the second time with default arguments. This type of bug is difficult to discover.

It is better to use `None` as a default and then have a check to see if it is still None (check first call) and only then do the assignment of the actual type. This assured the same behaviour each time the function is run.

The Hitchhiker's Guide to Python also has [a comprehensible example on mutable default arguments](https://github.com/StefanieSenger/Playground/blob/main/mutable_default_arguments.py).

#### Defensive Programming with Mutable Parameters
When coding a function that receives a mutable parameter, we should carefully consider whether the caller expects the argument passed to be changed, for instance receiving a dict: If our function modifies it while processing: should this side effect be visible outside of the function or not?

If object should be modified: comment on it (not do it implicitly), if object should not be modified: make a copy (copy.copy(obj) or copy.deepcopy(obj)).

### del and Garbage Collection
- `del` deletes references to objects, not the objects themselves
- nothing is explicitly deleted, only forgotten
- garbage collector may discard an object, if a del statement results in the object count going down to 0

In [14]:
a = [1, 2] # binding a to a list object
b = a  # binding b to the same lift object
del a # delete reference of a
print(b) # b's reference to the object is still there
b = [3] # this re-binds b to another list object, thus the reference tro [1, 2] can now be garbage collected

[1, 2]


We should avoid using the `__del__` special method in our code. The Python interpreter calls it if it needs to and modifying it can lead to issues.

In CPython, each object keeps count of how often it was referenced.

When `refcount` reaches 0, or CPython finds circular references, the interpreter calls the `__del__` special method and the object is forgotten.

We can use `weakreference`, if we don't want to increase the reference count. "Weak references are useful in caching applications because you don’t want the cached objects to be kept alive just because they are referenced by the cache." Or: a class that keeps track of all its current instances.

### Tricks Python Plays with Immutables
CPython uses an optimization technique called `interning` for small integers that are often used (`Beyond the Basis Stuff with Python` also talkes about it) and some other mutable objects, that will get the same ID, even if they are created independently:

In [15]:
s1 = 'ABC'
s2 = 'ABC'
s2 is s1

True

Which immutable types this refers to is undocumented and not be be relied on. It is only an internal optimization technique used by the current version of CPython interpreter to have some often used immutables at memory before the programme starts running.